## **Data Collection and Preparation Steps for Wildfire Risk Modeling**

**1️⃣ Fire History Data (Shapefiles)**

✅ **Dataset:** Historical Wildfire Perimeters  

📥 **Download Link:**  
- 🔗 **[CalFire Fire Perimeters (California)](https://data.ca.gov/dataset/california-historical-fire-perimeters)**

---

**2️⃣ Climate & Weather Data (Raster)**

✅ **Dataset:** Temperature, Humidity, Wind Speed (Daily Climate Data)

📥 **Download Link:**  
- 🔗 **[NASA POWER (Global Climate Data)](https://power.larc.nasa.gov/data-access-viewer/)**

---

**3️⃣ Terrain & Vegetation Data (Elevation, NDVI)**

✅ **Dataset:** NDVI (Vegetation Index) & Digital Elevation Model (DEM)

📥 **Download Link:**  

- 🔗 **[MODIS/Terra Vegetation Indices 16-Day L3 Global 250m SIN Grid V061-NDVI (NASA)](https://search.earthdata.nasa.gov/search)**
- 🔗 **[GEDI L2A Elevation and Height Metrics (NASA)](https://search.earthdata.nasa.gov/search)**

---

**4️⃣ Human Infrastructure Data (Roads, Urban Areas)**

✅ **Dataset:** Road Networks & Urban Boundaries  
📥 **Download Link:**  

- 🔗 **[California All Public Roads Network](https://gisdata-caltrans.opendata.arcgis.com/datasets/2d56e65de89c418780056651640291e8_0/about)**
- 🔗 **[Adjusted Urban Areas](https://gisdata-caltrans.opendata.arcgis.com/datasets/71930cd25e6f4683ba2ee18511ac21ef_0/about)**



**Spatial Clipping**

Datasets were clipped to a bounding box covering the specific region of California relevant to the modeling task:
*   **Longitude: -120.38 to -116.87**
*   **Latitude: 32.76 to 35.17**

**Coordinate Reference System (CRS) Alignment**

All geospatial data is standardized to the WGS84 geographic coordinate system (EPSG:4326). For accurate distance computations, a projected coordinate system (California Albers, EPSG:3310) is used.

In [ ]:
# Install Libraries
!pip install --quiet geopandas xarray pyhdf h5py

# Set up Google Drice
from google.colab import drive
drive.mount("/content/drive")

# Import Libraries
import sys
sys.path.append("/content/drive/MyDrive/Fire")
import data_loader
BASE_DIR = "/content/drive/MyDrive/Fire"

import os
import gc
import h5py
import glob
import xarray as xr

import pandas as pd
from tqdm import tqdm
import geopandas as gpd
from shapely import box
from pyhdf.SD import SD, SDC
from datetime import datetime
from scipy.spatial import cKDTree
from shapely.geometry import Point
from shapely.geometry import Polygon, MultiPolygon

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Climate & Weather Data**

In [ ]:
nc_files = glob.glob(os.path.join(BASE_DIR, "*.nc"))
humidity_path = [f for f in nc_files if "_h" in f][0]
temperature_path = [f for f in nc_files if "_t" in f][0]
wind_path = [f for f in nc_files if "_w" in f][0]

climate_df = data_loader.load_climate_data(humidity_path, temperature_path, wind_path)
climate_df.to_csv(os.path.join(BASE_DIR,"climate_df.csv"), index=False)

### **Vegetation Data (MODIS)**

In [ ]:
%cd /content
!chmod +x /content/drive/MyDrive/Fire/vegetation-filtered-download.sh

!mkdir -p vegetation
%cd vegetation
!bash /content/drive/MyDrive/Fire/vegetation-filtered-download.sh

In [ ]:
# Define MODIS NDVI directory
veg_dir = "/content/vegetation"
hdf_files = [os.path.join(veg_dir, f) for f in os.listdir(veg_dir) if f.endswith(".hdf")]

# Compute lat/lon grid once (Bounding box approach)
lat_min, lat_max = 32.76, 35.17
lon_min, lon_max = -120.38, -116.87

output_file = os.path.join(BASE_DIR,"ndvi_df.csv")
first_write = True  # Track first file to write headers

data_loader.extract_ndvi(hdf_files, lat_range=(32.76, 35.17), lon_range=(-120.38, -116.87), output_file=output_file)
!rm -r /content/vegetation

### **Elevation and Canopy Height (GEDI)**

In [ ]:
%cd /content/
!chmod +x /content/drive/MyDrive/Fire/elevation-download.sh

%mkdir -p elevation
%cd elevation
!bash /content/drive/MyDrive/Fire/elevation-download.sh

In [ ]:
gedi_dir = "/content/elevation"
gedi_files = [os.path.join(gedi_dir, f) for f in os.listdir(gedi_dir) if f.endswith(".h5")]

gedi_path = "/content/elevation/GEDI02_A_2023001145522_O22964_03_T09998_02_003_02_V002.h5"
elevation_df, canopy_df = data_loader.load_gedi_data(gedi_path, beam_name="BEAM0001")

elevation_df, canopy_df = data_loader.combine_gedi_datasets(gedi_files)
elevation_canopy_gdf = data_loader.spatial_merge_elevation_canopy(elevation_df, canopy_df)

elevation_canopy_gdf.to_csv(os.path.join(BASE_DIR, "canopy_elevation_gdf.csv"), index=False)
!rm -r /content/elevation

### **Fire History & Human Infrastructure (Roads, Urban Areas)**

In [ ]:
bbox = box(-120.38, 32.76, -116.87, 35.17)

fire_path = os.path.join(BASE_DIR, "California_Fire_Perimeters_(1950%2B).shp")
fires = data_loader.load_fire_data(fire_path, bbox)
fires = fires[fires["YEAR_"] == 2023.0]

road_path = os.path.join(BASE_DIR, "California_All_Public_Roads_Network.shp")
urban_area_path = os.path.join(BASE_DIR, "2020_Adjusted_Urban_Area.geojson")
roads, urban_area = data_loader.load_infrastructure_data(road_path, urban_area_path, bbox)

Compute Fire Distances to Urban Areas & Roads & Merge with Fire Data

In [ ]:
fires_df = data_loader.process_fire_data(fires, urban_area, roads)

columns = ['lat', 'lon','date', 'CONT_DATE', 'Shape__Are', 'Shape__Len', 'geometry', 'DISTANCE_TO_URBAN_AREAS', 'DISTANCE_TO_ROADS']
fires_df = fires_df[columns]
fires_df.to_csv(os.path.join(BASE_DIR, "fires_df.csv"), index=False)

### **Merging Data**


In [ ]:
fires_df = pd.read_csv(os.path.join(BASE_DIR, "fires_df.csv"))
climate_df = pd.read_csv(os.path.join(BASE_DIR, "climate_df.csv"))
ndvi_df = pd.read_csv(os.path.join(BASE_DIR, "ndvi_df.csv"))
elevation_canopy_df = pd.read_csv(os.path.join(BASE_DIR, "canopy_elevation_gdf.csv"))

fires_df = data_loader.merge_datasets(fires_df, climate_df, ndvi_df, elevation_canopy_df)

# Check the range of lat/lon to ensure correctness
print("Lat range:", fires_df["lat"].min(), "to", fires_df["lat"].max())
print("Lon range:", fires_df["lon"].min(), "to", fires_df["lon"].max())

fires_df.to_csv(os.path.join(BASE_DIR, "Fire_Risk_Data.csv"), index=False)